# Formulate the portfolio optimization problem and solve on CPU

This notebook demonstrates how to optimize a portfolio using CVaR on a CPU, we leverage CLARABEL solver from CVXPY to solve the optimization problem. 

The steps include:

1. **Configure CPU Solver Settings:** Define solver parameters for optimization
2. **Solve the CVaR Optimization Problem:** Run the optimization 
3. **Display the optimization results:** Examine the optimized portfolio
4. **Plot the optimized portfolio:** Visualize the allocation over the selected day range


## Importing Required Libraries

In [3]:
import os
from cufolio import cvar_optimizer, cvar_utils, utils, backtest, porfolio

import cvxpy as cp
import numpy as np
import pandas as pd
import seaborn as sns

import solver_settings
import warnings
warnings.filterwarnings("ignore")


ModuleNotFoundError: No module named 'cufolio'

## Formulating the Optimization Problem

$$ \max_{\mathbf{w}, t, \mathbf{u}} \quad \mu^\top \mathbf{w} - \lambda_{\text{risk}} \left( t + \frac{1}{1 - \alpha} \mathbf{p}^\top \mathbf{u} \right) $$

Where:

𝜇: vector of expected returns

𝑤 : portfolio weights

$𝜆_{risk}$: risk aversion coefficient

𝛼
: confidence level for CVaR

𝑝
: scenario probabilities

𝑡
: Value-at-Risk (VaR) threshold

𝑢
: CVaR auxiliary variables

The constraints are:
* Budget Constaint: $$\mathbf{1}^\top \mathbf{w} = 1$$
* CVaR auxiliary constraint: $$\mathbf{u} \geq -R \mathbf{w} - t$$
* Box constraints on weights and exposures: $$\mathbf{w}^{\min} \leq \mathbf{w} \leq \mathbf{w}^{\max}$$
$$\mathbf{c}^{\min} \leq \mathbf{c} \leq \mathbf{c}^{\max}$$
* Leverage constraint: $$\|\mathbf{w}\|_1 \leq L^{\text{tar}}$$

You can also account for a couple more additional constraints, including:

* CVar limit: setting a strict upper bound on the CVar
* Turnover constraint: restrict portfolio turnover by limiting deviations from existing portfolio

## Instantiate Parameters and Optimizer objects

Broadly, you want a high number of scenarios to make it perform optimally on a GPU
### Parameters

Portfolio Constraints

* `w_min`, `w_max`: Minimum and maximum asset weights
* `c_min`, `c_max`: Minimum and maximum cash holdings
* `L_tar`: Leverage constraint

Risk constraints:

* 





In [ ]:
cvar_params = cvar_optimizer.CVaR_Parameters(
w_min=-0.3, # Minimum asset weight allocation
w_max=0.8, # Maximum asset weight allocation
c_min=0.1, # Minimum cash holdings
c_max=0.4, # Maximum cash holdings
L_tar=1.6, # Leverage constraint (total long/short weight change)
T_tar=None, # Turnover constraint (Limits portfolio deviation from an existing allocation)
cvar_limit=None, # Maximum allowed Conditional Value-at-Risk (CVaR)
risk_aversion=0.5, # Change to risk aversion levels (0 = maximize return subject to risk constraints)
confidence=0.95, # Confidence level for CVaR calculation
num_scen=10000, # Number of return scenarios to simulate
fit_type= 'kde' # Return distribution estimation method ('kde' for kernel density estimation, 'no_fit' for historical returns)
)

### Loading Returns Data

In [ ]:
# Select dataset and regime information
dataset_name = 'sp500' # Dataset with adjusted close prices
dataset_directory = f'data/stock_data/{dataset_name}.csv'
# Define return type and regime period
return_type = 'LOG'
regime_dict = {'name': 'recent', 'range': ('2022-01-04', '2024-07-07')}
# Compute returns dictionary (set device for KDE computation)
returns_dict = cvar_utils.calculate_returns(
    dataset_directory, regime_dict, return_type, cvar_params, device='CPU'
)

NameError: name 'cvar_utils' is not defined

### Instantiate CVaR Optimizer
Finally, we create CVaR optimizer object, which integrates the computed return data and optimization parameters

In [ ]:
# Instantiate CVaR optimization problem
example_cvar_problem = cvar_optimizer.CVar(
returns_dict=returns_dict, 
cvar_params=cvar_params
)

## Portfolio Optimization Using CVaR on CPU

### Configure CPU solver settings and solve the CVaR optimization on CPU



The below code will output the optimized portfolio and suggest which trades to long and short in the test regime along with the objective value used to minimise relative risk. 

In [ ]:
# Configure CPU Solver Settings
cpu_settings = {'solver': cp.CLARABEL, 'verbose': False}
# Solve the CVaR Optimization Problem on CPU
cpu_result, cpu_portfolio = example_cvar_problem.solve_optimization_problem(device='CPU', cpu_settings=cpu_settings)